# Pattern Recognition - Apple Stock 2023-2025

Testing pattern recognition indicators: Pivots, Hammer, RSI Divergence.

In [1]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from pathlib import Path
sys.path.insert(0, str(Path('../..').resolve()))

from indicators.pattern import find_pivots, calculate_hammer, detect_rsi_divergence
from indicators.momentum import calculate_rsi

In [2]:
# Fetch Apple data
df = yf.download('AAPL', start='2023-01-01', end='2025-10-01', auto_adjust=True, progress=False)

# Handle MultiIndex columns from yfinance
if df.columns.nlevels == 2:
    df.columns = df.columns.get_level_values(0)

df.columns = df.columns.str.lower()
df = df.reset_index()
df.columns = df.columns.str.lower()

print(f"Data shape: {df.shape}")
df.head()

Data shape: (688, 6)


Price,date,close,high,low,open,volume
0,2023-01-03,123.330635,129.079551,122.443150,128.468178,112117500
1,2023-01-04,124.602715,126.870731,123.340517,125.125343,89113600
2,2023-01-05,123.281357,125.993113,123.024978,125.362014,80962700
3,2023-01-06,127.817337,128.478017,123.153123,124.257549,87754700
4,2023-01-09,128.339996,131.554669,128.083618,128.655553,70790800


## 1. Pivot Points

Identifies local highs and lows using lookback/lookahead windows (lb=8, rb=13). CRITICAL for ML labeling - marks turning points.

In [3]:
pivot_high, pivot_low = find_pivots(df, lb=8, rb=13, use_close=True)
df['PivotHigh'] = pivot_high
df['PivotLow'] = pivot_low

fig = go.Figure()

# Price line
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black', width=1)))

# Pivot highs
pivot_highs = df[df['PivotHigh'] == 1]
fig.add_trace(go.Scatter(x=pivot_highs['date'], y=pivot_highs['close'],
                        mode='markers', name='Pivot High',
                        marker=dict(color='red', size=10, symbol='triangle-down')))

# Pivot lows
pivot_lows = df[df['PivotLow'] == 1]
fig.add_trace(go.Scatter(x=pivot_lows['date'], y=pivot_lows['close'],
                        mode='markers', name='Pivot Low',
                        marker=dict(color='green', size=10, symbol='triangle-up')))

fig.update_layout(height=600, title_text="Pivot Points Detection (lb=8, rb=13)")
fig.show()

print(f"Pivot Highs found: {(df['PivotHigh'] == 1).sum()}")
print(f"Pivot Lows found: {(df['PivotLow'] == 1).sum()}")

Pivot Highs found: 17
Pivot Lows found: 25


## 1.1 Pivot Points with Window Variations

Extended pivot detection allowing ±1 and ±2 day tolerance around base pivots. Adjacent days marked as pivots only if close price within 5% of base pivot (price_tolerance default).

In [5]:
# Test different window variations
pivot_high_var1, pivot_low_var1 = find_pivots(df, lb=8, rb=13, use_close=True, 
                                               window_variations=[-1, 1])
pivot_high_var2, pivot_low_var2 = find_pivots(df, lb=8, rb=13, use_close=True, 
                                               window_variations=[-2, -1, 1, 2])

# Create comparison figure
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Base Pivots (no variation)', 
                                    'Pivots with ±1 day variation',
                                    'Pivots with ±2 day variation'))

# Row 1: Base pivots
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black', width=1), showlegend=False), row=1, col=1)
pivot_highs_base = df[df['PivotHigh'] == 1]
pivot_lows_base = df[df['PivotLow'] == 1]
fig.add_trace(go.Scatter(x=pivot_highs_base['date'], y=pivot_highs_base['close'],
                        mode='markers', name='Pivot High',
                        marker=dict(color='red', size=8, symbol='triangle-down')), row=1, col=1)
fig.add_trace(go.Scatter(x=pivot_lows_base['date'], y=pivot_lows_base['close'],
                        mode='markers', name='Pivot Low',
                        marker=dict(color='green', size=8, symbol='triangle-up')), row=1, col=1)

# Row 2: ±1 variation
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black', width=1), showlegend=False), row=2, col=1)
pivot_highs_var1 = df[pivot_high_var1 == 1]
pivot_lows_var1 = df[pivot_low_var1 == 1]
fig.add_trace(go.Scatter(x=pivot_highs_var1['date'], y=pivot_highs_var1['close'],
                        mode='markers', name='Pivot High ±1',
                        marker=dict(color='red', size=8, symbol='triangle-down'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=pivot_lows_var1['date'], y=pivot_lows_var1['close'],
                        mode='markers', name='Pivot Low ±1',
                        marker=dict(color='green', size=8, symbol='triangle-up'), showlegend=False), row=2, col=1)

# Row 3: ±2 variation
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black', width=1), showlegend=False), row=3, col=1)
pivot_highs_var2 = df[pivot_high_var2 == 1]
pivot_lows_var2 = df[pivot_low_var2 == 1]
fig.add_trace(go.Scatter(x=pivot_highs_var2['date'], y=pivot_highs_var2['close'],
                        mode='markers', name='Pivot High ±2',
                        marker=dict(color='red', size=8, symbol='triangle-down'), showlegend=False), row=3, col=1)
fig.add_trace(go.Scatter(x=pivot_lows_var2['date'], y=pivot_lows_var2['close'],
                        mode='markers', name='Pivot Low ±2',
                        marker=dict(color='green', size=8, symbol='triangle-up'), showlegend=False), row=3, col=1)

fig.update_layout(height=1600, title_text="Pivot Detection: Window Variations Comparison (5% price tolerance)")
fig.show()

print(f"Base Pivots - Highs: {(df['PivotHigh'] == 1).sum()}, Lows: {(df['PivotLow'] == 1).sum()}")
print(f"±1 Variation - Highs: {(pivot_high_var1 == 1).sum()}, Lows: {(pivot_low_var1 == 1).sum()}")
print(f"±2 Variation - Highs: {(pivot_high_var2 == 1).sum()}, Lows: {(pivot_low_var2 == 1).sum()}")
print("\nWindow variations expand pivot labels for ML training, but only if adjacent day's close is within 5% of base pivot.")

Base Pivots - Highs: 17, Lows: 25
±1 Variation - Highs: 36, Lows: 49
±2 Variation - Highs: 46, Lows: 59

Window variations expand pivot labels for ML training, but only if adjacent day's close is within 5% of base pivot.


## 2. Hammer Candlestick Pattern

Bullish reversal pattern: small body at top, long lower shadow (>2x body), little/no upper shadow. Signals buying pressure overcame selling.

In [6]:
df['hammer'] = calculate_hammer(df)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price with Hammer Patterns', 'Hammer Signal'))

# Price
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black')), row=1, col=1)

# Mark hammer patterns on price
hammers = df[df['hammer'] == 100]
fig.add_trace(go.Scatter(x=hammers['date'], y=hammers['close'],
                        mode='markers', name='Hammer',
                        marker=dict(color='orange', size=10, symbol='star')), row=1, col=1)

# Hammer signal
fig.add_trace(go.Scatter(x=df['date'], y=df['hammer'], name='Hammer',
                        fill='tozeroy', line=dict(color='orange')), row=2, col=1)

fig.update_layout(height=600, title_text="Hammer Candlestick Pattern (Bullish Reversal)")
fig.show()

print(f"Hammer patterns found: {(df['hammer'] == 100).sum()}")

Hammer patterns found: 17


## 3. RSI Divergence at Pivot Lows

Bullish divergence: price makes lower low but RSI makes higher low at consecutive pivot points. Suggests momentum turning bullish.

In [7]:
# Calculate RSI first
df['RSI'] = calculate_rsi(df, period=14)

# Detect divergence
df = detect_rsi_divergence(df)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price with Divergence', 'RSI', 'Divergence Signal'))

# Price with pivot lows
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black')), row=1, col=1)
pivot_lows = df[df['PivotLow'] == 1]
fig.add_trace(go.Scatter(x=pivot_lows['date'], y=pivot_lows['close'],
                        mode='markers', name='Pivot Low',
                        marker=dict(color='blue', size=8)), row=1, col=1)

# Mark divergence points
divergences = df[df['RSI_Divergence'] == 1]
fig.add_trace(go.Scatter(x=divergences['date'], y=divergences['close'],
                        mode='markers', name='Bullish Divergence',
                        marker=dict(color='green', size=12, symbol='star')), row=1, col=1)

# RSI
fig.add_trace(go.Scatter(x=df['date'], y=df['RSI'], name='RSI',
                        line=dict(color='blue')), row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="red", row=2, col=1)

# Divergence signal
fig.add_trace(go.Scatter(x=df['date'], y=df['RSI_Divergence'], name='Divergence',
                        fill='tozeroy', line=dict(color='green')), row=3, col=1)

fig.update_layout(height=800, title_text="RSI Bullish Divergence Detection")
fig.show()

print(f"Bullish divergences found: {(df['RSI_Divergence'] == 1).sum()}")
print("Bullish divergence = Lower price but Higher RSI at pivot lows")

Bullish divergences found: 2
Bullish divergence = Lower price but Higher RSI at pivot lows


## Combined View - All Pattern Indicators

In [8]:
fig = go.Figure()

# Price line
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black', width=1.5)))

# Pivot highs (red triangles)
pivot_highs = df[df['PivotHigh'] == 1]
fig.add_trace(go.Scatter(x=pivot_highs['date'], y=pivot_highs['close'],
                        mode='markers', name='Pivot High',
                        marker=dict(color='red', size=10, symbol='triangle-down')))

# Pivot lows (green triangles)
pivot_lows = df[df['PivotLow'] == 1]
fig.add_trace(go.Scatter(x=pivot_lows['date'], y=pivot_lows['close'],
                        mode='markers', name='Pivot Low',
                        marker=dict(color='green', size=10, symbol='triangle-up')))

# Hammer patterns (orange stars)
hammers = df[df['hammer'] == 100]
fig.add_trace(go.Scatter(x=hammers['date'], y=hammers['close'],
                        mode='markers', name='Hammer',
                        marker=dict(color='orange', size=12, symbol='star')))

# RSI divergences (large green circles)
divergences = df[df['RSI_Divergence'] == 1]
fig.add_trace(go.Scatter(x=divergences['date'], y=divergences['close'],
                        mode='markers', name='RSI Divergence',
                        marker=dict(color='lime', size=15, symbol='circle',
                                  line=dict(color='darkgreen', width=2))))

fig.update_layout(height=700, title_text="Pattern Recognition - Complete Overview",
                 xaxis_title="Date", yaxis_title="Price")
fig.show()

print("\n=== Pattern Summary ===")
print(f"Pivot Highs: {(df['PivotHigh'] == 1).sum()}")
print(f"Pivot Lows: {(df['PivotLow'] == 1).sum()}")
print(f"Hammer Patterns: {(df['hammer'] == 100).sum()}")
print(f"RSI Bullish Divergences: {(df['RSI_Divergence'] == 1).sum()}")
print("\nPivots identify local extremes for ML labeling.")
print("Hammers signal potential reversals.")
print("Divergences show momentum/price disagreement.")


=== Pattern Summary ===
Pivot Highs: 17
Pivot Lows: 25
Hammer Patterns: 17
RSI Bullish Divergences: 2

Pivots identify local extremes for ML labeling.
Hammers signal potential reversals.
Divergences show momentum/price disagreement.
